<a href="https://colab.research.google.com/github/yiyichanmyae/tensorflow/blob/master/2_3_human_vs_horse_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import urllib.request
import zipfile

def getDataFromUrl(url_, file_location):
  urllib.request.urlretrieve(url_, file_location)

def extractZip(zip_, folder_dir):
  zip_ref = zipfile.ZipFile(zip_, "r")
  zip_ref.extractall(folder_dir)
  zip_ref.close()

# load data
training_file_name = "h-or-h-train.zip"
training_dir = "h-or-h/training"
validation_file_name = "h-or-h-validation.zip"
validation_dir = "h-or-h/vlidation"

training_url = "https://storage.googleapis.com/learning-datasets/horse-or-human.zip"
getDataFromUrl(training_url, training_file_name)
extractZip(training_file_name, training_dir)

# urllib.request.urlretrieve(training_url, training_file_name)
# zip_ref = zipfile.ZipFile(training_file_name, "r")
# zip_ref.extractall(training_dir)
# zip_ref.close()

validation_url = "https://storage.googleapis.com/learning-datasets/validation-horse-or-human.zip"
getDataFromUrl(validation_url, validation_file_name)
extractZip(validation_file_name, validation_dir)

# urllib.request.urlretrieve(validation_url, validation_file_name)
# zip_ref = zipfile.ZipFile(validation_file_name, "r")
# zip_ref.extractall(validation_dir)
# zip_ref.close()

from tensorflow.keras.preprocessing.image import ImageDataGenerator

